In [1]:
# import basic machine learning libraries 

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sea 

In [43]:
import warnings
warnings.filterwarnings('ignore')

In [93]:
from sklearn.model_selection import train_test_split

In [118]:
from sklearn.metrics import accuracy_score

In [102]:
# import tensorflow integrated with keras libraries 

import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding,Dense,Flatten
from tensorflow.keras.models import Sequential

In [25]:
# import nltk libraies for text preprocessing 

import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
import re

In [32]:
stop_words=stopwords.words('english')
stemmer=PorterStemmer()
lemitizer=WordNetLemmatizer()

In [8]:
df=pd.read_csv('train.csv')

In [9]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
df.shape

(20800, 5)

In [11]:
df_null=(df.isnull().sum()/df.shape[0])*100

In [12]:
df_null.sort_values(ascending=False)

author    9.408654
title     2.682692
text      0.187500
id        0.000000
label     0.000000
dtype: float64

In [13]:
df=df.dropna()

In [14]:
df.shape

(18285, 5)

In [15]:
y=df['label']

In [16]:
df['label'].value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [19]:
df['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [26]:
import re

def clean_text(text):
 
    
    
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)    
    text = text.strip().lower()
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text

In [28]:
df['title']=df['title'].apply(lambda x:clean_text(x))

In [29]:
df.head()

,id,title,author,text,label
0,0,house dem aide we didn’t even see comey’s let...,Darrell Lucus,house dem aide we didn’t even see comey’s let...,1
1,1,flynn hillary clinton big woman on campus ...,Daniel J. Flynn,ever get the feeling your life circles the rou...,0
2,2,why the truth might get you fired,Consortiumnews.com,why the truth might get you fired october 29 ...,1
3,3,15 civilians killed in single us airstrike hav...,Jessica Purkiss,videos 15 civilians killed in single us airstr...,1
4,4,iranian woman jailed for fictional unpublished...,Howard Portnoy,print an iranian woman has been sentenced to ...,1


In [44]:
df_final=df[['title','label']]

In [45]:
df_final.head()

,title,label
0,house dem aide we didn’t even see comey’s let...,1
1,flynn hillary clinton big woman on campus ...,0
2,why the truth might get you fired,1
3,15 civilians killed in single us airstrike hav...,1
4,iranian woman jailed for fictional unpublished...,1


In [46]:
def cleaning(text):
    var1=[]
    for i in word_tokenize(text):
        var1.append(i)
    var2=[]
    for i in var1:
        if i not in stop_words:
            var2.append(i)
    var3=[]
    for i in var2:
        var3.append(stemmer.stem(i))
    var4=[]
    for i in var3:
        var4.append(lemitizer.lemmatize(i))
        
    return var4

In [47]:
df_final['title']=df_final['title'].apply(lambda x:cleaning(x))

In [79]:
df_final.head(10)

,title,label
0,hous dem aid ’ even see comey ’ letter jason c...,1
1,flynn hillari clinton big woman campu breitbart,0
2,truth might get fire,1
3,15 civilian kill singl u airstrik identifi,1
4,iranian woman jail fiction unpublish stori wom...,1
5,jacki mason hollywood would love trump bomb no...,0
6,benoît hamon win french socialist parti ’ pres...,0
7,back channel plan ukrain russia courtesi trump...,0
8,obama ’ organ action partner soro link ‘ indiv...,0
9,bbc comedi sketch real housew isi caus outrag,0


In [78]:
df_final['title']=df_final['title'].apply(lambda x:' '.join(x))

In [56]:
df_final=df_final.reset_index(drop=True)

In [57]:
len(df['title'][0])

81

In [69]:
var1=0
for i in range(len(df_final)):
    var1=var1+len(df_final['title'][i])

In [70]:
var1

176419

In [71]:
var1/df_final.shape[0]

9.64829094886519

In [72]:
sent_length=15

In [80]:
corpus=[]
for i in range(len(df_final)):
    corpus.append(df_final['title'][i])

In [81]:
corpus

['hous dem aid ’ even see comey ’ letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 '15 civilian kill singl u airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'benoît hamon win french socialist parti ’ presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama ’ organ action partner soro link ‘ indivis ’ disrupt trump ’ agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base ‘ treasur hunter ’ arctic photo',
 'u offici see link trump russia',
 'ye paid govern troll social medium blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay 2 5 million releas everyon arrest dakota access p

In [75]:
dictionary=10000

In [82]:
onehot_corpus=[one_hot(word,dictionary)for word in corpus]

In [83]:
onehot_corpus

[[1842, 5365, 5603, 8662, 2615, 4988, 5442, 8662, 5831, 5851, 7406, 5799],
 [2382, 7042, 2567, 1447, 4375, 4057, 1851],
 [1083, 7121, 3118, 6992],
 [9962, 7721, 3168, 480, 1547, 86, 2657],
 [1145, 4375, 2169, 7150, 9482, 3040, 4375, 2214, 5296, 8],
 [9039,
  2950,
  8128,
  1795,
  5249,
  9066,
  4013,
  1516,
  5155,
  723,
  1079,
  8094,
  8586,
  8471,
  1851],
 [6930, 6219, 4134, 7674, 7839, 7866, 8662, 1736, 225, 2293, 3386, 5947],
 [5596, 3925, 7019, 2281, 6737, 6706, 9066, 623, 2293, 3386, 5947],
 [4287,
  8662,
  5905,
  7226,
  9296,
  5067,
  7177,
  227,
  8831,
  8662,
  4307,
  9066,
  8662,
  8173],
 [8293, 9109, 9568, 8829, 9582, 3941, 8612, 1163],
 [6356, 5981, 6393, 9916, 180, 4885, 2694, 227, 4287, 8476, 8662, 863, 3838],
 [1547, 1370, 4988, 7177, 9066, 6737],
 [1638, 6469, 7023, 2109, 4953, 8583, 4886, 3313, 8591],
 [1036, 665, 4962, 427, 8655, 8192, 3743, 2293, 3386, 5947],
 [8705, 4934, 1313, 5536, 7158, 2293, 3386, 5947],
 [2463, 9912, 7747, 2400, 8229, 146, 501

In [84]:
embedded_doc=pad_sequences(onehot_corpus,maxlen=20,padding='pre')

In [85]:
embedded_doc

array([[   0,    0,    0, ..., 5851, 7406, 5799],
       [   0,    0,    0, ..., 4375, 4057, 1851],
       [   0,    0,    0, ..., 7121, 3118, 6992],
       ...,
       [   0,    0,    0, ..., 2293, 3386, 5947],
       [   0,    0,    0, ...,  774, 8964, 7133],
       [   0,    0,    0, ..., 3974, 2289, 5062]])

In [86]:
embedding_vector_feature=50

In [103]:
model=Sequential()

model.add(Embedding(dictionary,embedding_vector_feature,input_length=20))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [104]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 50)            500000    
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 501,001
Trainable params: 501,001
Non-trainable params: 0
_________________________________________________________________


In [105]:
embedded_doc.shape

(18285, 20)

In [106]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [107]:
x_final=np.array(embedded_doc)
y_final=np.array(y)

In [108]:
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.20)

In [109]:
model.fit(x_train,y_train,epochs=10,batch_size=64)

Epoch 1/10
229/229 [==============================] - 12s 21ms/step - loss: 0.3822 - accuracy: 0.8466
Epoch 2/10
229/229 [==============================] - 3s 14ms/step - loss: 0.1477 - accuracy: 0.9454
Epoch 3/10
229/229 [==============================] - 3s 12ms/step - loss: 0.0874 - accuracy: 0.9725
Epoch 4/10
229/229 [==============================] - 3s 13ms/step - loss: 0.0535 - accuracy: 0.9869
Epoch 5/10
229/229 [==============================] - 3s 13ms/step - loss: 0.0328 - accuracy: 0.9946
Epoch 6/10
229/229 [==============================] - 3s 13ms/step - loss: 0.0204 - accuracy: 0.9977
Epoch 7/10
229/229 [==============================] - 3s 14ms/step - loss: 0.0130 - accuracy: 0.9992
Epoch 8/10
229/229 [==============================] - 3s 13ms/step - loss: 0.0086 - accuracy: 0.9997
Epoch 9/10
229/229 [==============================] - 3s 13ms/step - loss: 0.0059 - accuracy: 0.9999
Epoch 10/10
229/229 [==============================] - 3s 13ms/step - loss: 0.0042 - accur

In [111]:
y_pred=model.predict(x_test)

In [112]:
df_final_pred=pd.DataFrame({'y_test':y_test,'y_pred':y_pred.flatten()})

In [117]:
df_final_pred.head()

,y_test,y_pred
0,0,0
1,0,0
2,1,1
3,0,0
4,0,0


In [115]:
def value(x):
    if x>0.5:
        return 1
    else:
        return 0

In [116]:
df_final_pred['y_pred']=df_final_pred['y_pred'].apply(lambda x:value(x))

In [119]:
accuracy_score(df_final_pred['y_test'],df_final_pred['y_pred'])

0.9398414000546896